In [ ]:
import pandas as pd
from pathlib import Path
import pymc as pm
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from estival.wrappers import pymc as epm
# from estival.wrappers import nevergrad as eng
# from estival.sampling import tools as esamp
# from estival.utils.parallel import map_parallel
import multiprocessing as mp
import nevergrad as ng
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import load_params, matrix, conmat
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,
    latent_compartments,
    infectious_compartments,
)
from tbdynamics.plotting import plot_model_vs_actual
import arviz as az

## Define variables

In [ ]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / "data"
OUT_PATH = PROJECT_PATH / "output"
pd.options.plotting.backend = "plotly"

time_start = 1800.0
time_end = 2023.0
time_step = 0.1
matrix_homo = np.ones((6, 6))  # Homo mixing

In [ ]:
fixed_parameters = load_params(PROJECT_PATH / "tbdynamics/params.yml")

## Define Model

### Base model

In [ ]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    conmat,
)

### Params and calibration targets

In [ ]:
params = {
    'smear_positive_death_rate': 0.339,
    'smear_negative_death_rate': 0.025,
    'smear_positive_self_recovery': 0.0192,
    'smear_negative_self_recovery': 0.073,
    'seed_num': 1.0, 
    'seed_duration': 1.0,
    'progression_multiplier':1.0
}


priors = [
    esp.UniformPrior("start_population_size", (2000000, 10000000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.2)),  
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (1.0, 5.0)),
    esp.UniformPrior("seed_time", (1890.0, 1950.0)),
    esp.UniformPrior("seed_num", (1.0, 100.00)),
    esp.UniformPrior("seed_duration", (1.0, 5.0)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,
    }
)

notifs = pd.Series(
    {
        1986: 8737,
        2010: 99022,
        2011: 100518,
        2012: 103906,
        2013: 102196,
        2014: 102087,
        2015: 102676,
        2016: 102527,
        2017: 105733,
        2018: 102171,
        2019: 104505,
        2020: 101795,
    }
)

prop_smear_positive = pd.Series({2022: 0.83})
incidence = pd.Series({2022: 176})
percentage_latent= pd.Series({2022: 43.0})

targets = [
    est.NormalTarget("total_population", pop, stdev=100000.0),
    est.NormalTarget("notification", notifs, stdev=1000.0),
    est.NormalTarget("prop_smear_positive", prop_smear_positive, stdev=0.5),
    est.NormalTarget("incidence", incidence, stdev=33.0),
    est.NormalTarget("percentage_latent", percentage_latent, stdev=5.0),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [ ]:
with pm.Model() as pmc_model:
    start_params = {
        k: np.clip(v, *calibration_model.priors[k].bounds(0.99))
        for k, v in params.items()
        if k in calibration_model.priors
    }
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(
        start=start_params, vars=variables, maxeval=20000, include_transformed=False
    )
    map_params = {k: float(v) for k, v in map_params.items()}
    print("Best calibration parameters found:")
for i_param, param in enumerate(map_params):
    print(
        f"   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}"
    )

In [ ]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()


### Population output

In [ ]:
plot_model_vs_actual(
    derived_df_0, pop, "total_population", "Population", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, incidence, "incidence", "Incidence", "Modelled vs Data"
)

In [ ]:
derived_df_0["prevalence_infectious"].plot()

In [ ]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [ ]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [ ]:
derived_df_0[[f"prop_{organ_stratum}" for organ_stratum in organ_strata]].plot(
    kind="area"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, notifs, "notification", "Notification", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, percentage_latent, "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"cdr_{organ_stratum}" for organ_stratum in organ_strata]].plot()

In [ ]:
# with pm.Model() as pm_model:
#     variables = epm.use_model(calibration_model)
#     idata_raw = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=5000, tune=1000, discard_tuned_samples=True)

In [ ]:
#az.plot_trace(idata_raw)

In [ ]:
# def calibrate(out_path, bcm, draws, tune):
#     def get_acceptable_start_params(n_params_target, ci=1.0):
#         params = []
#         n_cores = mp.cpu_count()
#         while len(params) < n_params_target:
#             new_samples = bcm.sample.lhs(n_cores).convert('list_of_dicts')
#             lle = esamp.likelihood_extras_for_samples(new_samples, bcm)
#         return bcm.sample.convert(params)

#     CI = 0.67
#     start_lhs = get_acceptable_start_params(8, ci=CI)
    
#     def optimize_ng(idx_sample):
#         idx, sample = idx_sample
#         opt = eng.optimize_model(bcm, budget=100, opt_class=ng.optimizers.TwoPointsDE, obj_function=bcm.logposterior, suggested=sample, num_workers=4, ci=CI)
#         rec = opt.minimize(100)
#         return idx, rec.value[1]

#     opt_samples = map_parallel(optimize_ng, start_lhs.iterrows(), n_workers=2, mode='process')
#     opt_samples = bcm.sample.convert(opt_samples).iloc[0: 8].convert('list_of_dicts')
    
#     n_chains = 8
#     n_samples = 100
#     with pm.Model() as pm_model:
#         variables = epm.use_model(bcm)
#         idata_raw = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=draws, tune=tune, cores=4, discard_tuned_samples=False, chains=n_chains, progressbar=True, initvals=opt_samples)
#     idata_raw.to_netcdf(str(out_path / 'calib_full_out.nc'))
    
#     burnt_idata = idata_raw.sel(draw=np.s_[2000:])
#     idata_extract = az.extract(burnt_idata, num_samples=n_samples)
    
#     bcm.sample.convert(idata_extract).to_hdf5(out_path / 'calib_extract_out.h5')
    
#     spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)
#     spaghetti_res.results.to_hdf(str(out_path / 'results.hdf'), 'spaghetti')

#     like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
#     like_df.to_hdf(str(out_path / 'results.hdf'), 'likelihood')

In [ ]:
# draws = 10000
# tune = 2000
# def run_calibration():
#     idata_raw = calibrate(OUT_PATH,calibration_model, draws,tune)


In [ ]:
#run_calibration()